# Income-Based Options Strategy

**Goal**: Build systematic wealth through options strategies, powered by $128k salary income.

## Strategy Framework

1. **Income Allocation** - How much to invest monthly
2. **Mispricing Scanner** - Find undervalued options
3. **Volatility Analysis** - Identify vol regime opportunities
4. **Strategy Selection** - Choose optimal strategies
5. **Portfolio Tracking** - Monitor and adjust

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Qwen imports
from qwen.data import YahooDataProvider
from qwen.pricing import BlackScholes
from qwen.portfolio.allocator import IncomeBasedAllocator, IncomeProfile, create_profile_128k
from qwen.screener.mispricing import MispricingScanner
from qwen.screener.volatility import VolatilityAnalyzer

# Initialize
provider = YahooDataProvider()

print("Setup complete!")

---
## 1. Income Profile & Allocation

Based on $128k salary at BlackRock, let's calculate monthly investable amount.

In [ ]:
# Create income profile
profile = IncomeProfile(
    annual_salary=128000,
    tax_bracket=0.32,          # 32% federal marginal
    state_tax=0.06,            # NY/CA
    retirement_contribution_pct=0.10,  # 10% to 401k
    emergency_months=6,
    monthly_expenses=4500,     # Higher for HCOL area
    existing_investments=0,
    risk_tolerance="moderate"
)

print("=" * 50)
print("INCOME PROFILE")
print("=" * 50)
print(f"Annual Salary:      ${profile.annual_salary:,.0f}")
print(f"401k Contribution:  ${profile.annual_salary * profile.retirement_contribution_pct:,.0f} ({profile.retirement_contribution_pct*100:.0f}%)")
print(f"Effective Tax Rate: {profile.effective_tax_rate*100:.1f}%")
print(f"Annual Take-Home:   ${profile.annual_take_home:,.0f}")
print(f"Monthly Take-Home:  ${profile.monthly_take_home:,.0f}")
print(f"Monthly Expenses:   ${profile.monthly_expenses:,.0f}")
print(f"Monthly Investable: ${profile.monthly_investable:,.0f}")
print(f"Annual Investable:  ${profile.monthly_investable * 12:,.0f}")

In [ ]:
# Calculate allocation
allocator = IncomeBasedAllocator(profile)
allocation = allocator.calculate_allocation()

print(allocation.summary())

print("\nStrategy Breakdown:")
for strategy, amount in allocation.strategies.items():
    print(f"  {strategy}: ${amount:,.0f}")

In [ ]:
# Project 10-year growth
projection = allocator.project_growth(years=10, annual_return=0.12)

print("10-Year Portfolio Projection (12% return)")
print("=" * 60)
print(f"Monthly Contribution: ${projection['monthly_contribution']:,.0f}")
print()

df_proj = pd.DataFrame(projection['projections'])
print(df_proj.to_string(index=False, formatters={
    'portfolio_value': '${:,.0f}'.format,
    'total_contributed': '${:,.0f}'.format,
    'total_gains': '${:,.0f}'.format,
    'gain_pct': '{:.1f}%'.format,
}))

print(f"\nFinal Portfolio Value: ${projection['final_value']:,.0f}")

In [ ]:
# Visualize growth
fig, ax = plt.subplots(figsize=(10, 6))

years = [p['year'] for p in projection['projections']]
values = [p['portfolio_value'] for p in projection['projections']]
contributed = [p['total_contributed'] for p in projection['projections']]

ax.fill_between(years, 0, contributed, alpha=0.3, label='Contributions')
ax.fill_between(years, contributed, values, alpha=0.3, label='Gains')
ax.plot(years, values, 'b-', linewidth=2, label='Total Value')

ax.set_xlabel('Year')
ax.set_ylabel('Portfolio Value ($)')
ax.set_title('10-Year Portfolio Projection')
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}k'))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 2. Volatility Regime Analysis

Understanding the current vol environment helps choose strategies:
- **Low vol**: Buy options (straddles, long calls/puts)
- **High vol**: Sell options (iron condors, covered calls)

In [ ]:
# Analyze volatility across watchlist
vol_analyzer = VolatilityAnalyzer(provider)

watchlist = ['SPY', 'QQQ', 'AAPL', 'MSFT', 'NVDA', 'AMD', 'TSLA', 'META', 'GOOGL', 'AMZN']

print("Analyzing volatility regimes...")
vol_df = vol_analyzer.find_vol_opportunities(watchlist)

if not vol_df.empty:
    print("\nVolatility Analysis:")
    display_cols = ['symbol', 'current_iv', 'realized_20d', 'iv_rv_ratio', 'iv_percentile', 'regime']
    print(vol_df[display_cols].to_string(index=False, formatters={
        'current_iv': '{:.1%}'.format,
        'realized_20d': '{:.1%}'.format,
        'iv_rv_ratio': '{:.2f}'.format,
        'iv_percentile': '{:.0f}'.format,
    }))

In [ ]:
# Identify cheap vol (buy) and expensive vol (sell) opportunities
if not vol_df.empty:
    cheap_vol = vol_df[vol_df['iv_percentile'] < 25].copy()
    expensive_vol = vol_df[vol_df['iv_percentile'] > 75].copy()
    
    print("=" * 50)
    print("CHEAP VOL (Consider buying options)")
    print("=" * 50)
    if not cheap_vol.empty:
        for _, row in cheap_vol.iterrows():
            print(f"  {row['symbol']}: IV {row['current_iv']*100:.0f}% (percentile: {row['iv_percentile']:.0f})")
            print(f"    -> {row['recommendation']}")
    else:
        print("  No cheap vol opportunities found")
    
    print("\n" + "=" * 50)
    print("EXPENSIVE VOL (Consider selling options)")
    print("=" * 50)
    if not expensive_vol.empty:
        for _, row in expensive_vol.iterrows():
            print(f"  {row['symbol']}: IV {row['current_iv']*100:.0f}% (percentile: {row['iv_percentile']:.0f})")
            print(f"    -> {row['recommendation']}")
    else:
        print("  No expensive vol opportunities found")

---
## 3. Options Mispricing Scanner

Scan for:
1. **Put-Call Parity Violations** - Arbitrage opportunities
2. **IV vs Realized Discrepancies** - Cheap/expensive options
3. **Skew Anomalies** - Unusual strike pricing

In [ ]:
# Initialize scanner
scanner = MispricingScanner(
    data_provider=provider,
    risk_free_rate=0.05,
    min_edge_pct=5.0,   # Minimum 5% edge to flag
    min_volume=100,     # Minimum option volume
)

# Scan watchlist
print("Scanning for mispricing opportunities...")
print("This may take a few minutes...\n")

opportunities_df = scanner.scan_watchlist(watchlist)

In [ ]:
# Display opportunities
if not opportunities_df.empty:
    print(f"\nFound {len(opportunities_df)} potential opportunities:\n")
    
    # Show top opportunities
    actionable = opportunities_df[opportunities_df['actionable'] == True]
    
    if not actionable.empty:
        print("=" * 70)
        print("TOP ACTIONABLE OPPORTUNITIES")
        print("=" * 70)
        print(actionable.head(10).to_string(index=False))
    else:
        print("No high-confidence opportunities found. Showing all:")
        print(opportunities_df.head(10).to_string(index=False))
else:
    print("No mispricing opportunities found.")
    print("This could mean:")
    print("  1. Markets are efficiently priced")
    print("  2. Low liquidity in options chains")
    print("  3. Thresholds are too strict")

---
## 4. Current Market Analysis

Let's look at specific opportunities based on current market conditions.

In [ ]:
# Get current prices and calculate key metrics
end = datetime.now()
start = end - timedelta(days=60)

analysis = []

for symbol in watchlist:
    try:
        hist = provider.get_historical(symbol, start, end)
        if hist.empty:
            continue
        
        price = hist['Close'].iloc[-1]
        returns = hist['Close'].pct_change().dropna()
        
        # Calculate metrics
        realized_vol = returns.tail(20).std() * np.sqrt(252)
        move_5d = (hist['Close'].iloc[-1] / hist['Close'].iloc[-5] - 1)
        move_20d = (hist['Close'].iloc[-1] / hist['Close'].iloc[-20] - 1)
        
        # 52-week high/low
        high_52w = hist['High'].max()
        low_52w = hist['Low'].min()
        pct_from_high = (price - high_52w) / high_52w
        
        analysis.append({
            'symbol': symbol,
            'price': price,
            '5d_move': move_5d,
            '20d_move': move_20d,
            'realized_vol': realized_vol,
            'from_52w_high': pct_from_high,
        })
    except Exception as e:
        print(f"Error with {symbol}: {e}")

analysis_df = pd.DataFrame(analysis)
print("\nMarket Analysis:")
print(analysis_df.to_string(index=False, formatters={
    'price': '${:.2f}'.format,
    '5d_move': '{:+.1%}'.format,
    '20d_move': '{:+.1%}'.format,
    'realized_vol': '{:.0%}'.format,
    'from_52w_high': '{:+.1%}'.format,
}))

---
## 5. Strategy Recommendations

Based on the analysis, here are recommended trades for this month.

In [ ]:
# Monthly investment plan
plan = allocator.get_monthly_plan()

print("=" * 60)
print(f"MONTHLY INVESTMENT PLAN - Month {plan['month']}")
print("=" * 60)
print(f"\nTotal to Invest: ${plan['total_to_invest']:,.0f}")
print("\nActions:")

for i, action in enumerate(plan['actions'], 1):
    print(f"\n{i}. {action['type'].upper()}")
    print(f"   Amount: ${action['amount']:,.0f}")
    print(f"   Description: {action['description']}")
    print(f"   Timing: {action['timing']}")

In [ ]:
# Generate specific trade ideas based on vol analysis
print("=" * 60)
print("SPECIFIC TRADE IDEAS")
print("=" * 60)

options_budget = allocation.options_allocation

if not vol_df.empty:
    # Find best candidates
    low_vol = vol_df[vol_df['iv_percentile'] < 30].head(2)
    high_vol = vol_df[vol_df['iv_percentile'] > 70].head(2)
    
    trade_ideas = []
    
    # Long vol trades
    for _, row in low_vol.iterrows():
        trade_ideas.append({
            'symbol': row['symbol'],
            'strategy': 'Long Straddle',
            'rationale': f"IV at {row['iv_percentile']:.0f} percentile - cheap options",
            'allocation': options_budget * 0.15,
        })
    
    # Short vol trades
    for _, row in high_vol.iterrows():
        trade_ideas.append({
            'symbol': row['symbol'],
            'strategy': 'Iron Condor',
            'rationale': f"IV at {row['iv_percentile']:.0f} percentile - expensive options",
            'allocation': options_budget * 0.20,
        })
    
    # Display trades
    for trade in trade_ideas:
        print(f"\n{trade['symbol']} - {trade['strategy']}")
        print(f"  Rationale: {trade['rationale']}")
        print(f"  Allocation: ${trade['allocation']:,.0f}")
else:
    print("\nNo vol-based trade ideas generated.")
    print("Consider:")
    print("  1. Cash-secured puts on quality stocks")
    print("  2. Covered calls on existing positions")
    print("  3. Calendar spreads for theta income")

---
## 6. Options Pricing Examples

Using our pricing models to evaluate specific trades.

In [ ]:
# Example: Evaluate a cash-secured put
symbol = 'AAPL'
hist = provider.get_historical(symbol, start, end)
spot = hist['Close'].iloc[-1]

# 5% OTM put, 30 days to expiry
strike = round(spot * 0.95, 0)
days = 30
vol = 0.25  # Estimated IV

bs = BlackScholes(spot, strike, 0.05, vol, days/365)

put_premium = bs.put_price()
put_delta = bs.delta('put')
greeks = bs.greeks('put')

print(f"CASH-SECURED PUT ANALYSIS: {symbol}")
print("=" * 50)
print(f"Current Price: ${spot:.2f}")
print(f"Put Strike:    ${strike:.0f} ({(strike/spot-1)*100:.1f}% OTM)")
print(f"Days to Exp:   {days}")
print(f"\nPut Premium:   ${put_premium:.2f}/share")
print(f"Cash Required: ${strike * 100:,.0f}")
print(f"Premium Recv:  ${put_premium * 100:.0f}")
print(f"Return on Cash: {put_premium / strike * 100:.2f}% ({put_premium / strike * 12 * 100:.1f}% annualized)")
print(f"\nGreeks:")
print(f"  Delta: {greeks.delta:.3f}")
print(f"  Gamma: {greeks.gamma:.4f}")
print(f"  Theta: ${greeks.theta:.3f}/day")
print(f"  Vega:  ${greeks.vega:.3f}/1% vol")

In [ ]:
# Example: Evaluate an iron condor
symbol = 'SPY'
hist = provider.get_historical(symbol, start, end)
spot = hist['Close'].iloc[-1]

# Iron condor strikes
short_put = round(spot * 0.96, 0)
long_put = short_put - 5
short_call = round(spot * 1.04, 0)
long_call = short_call + 5

days = 30
vol = 0.15

# Calculate each leg
bs_sp = BlackScholes(spot, short_put, 0.05, vol, days/365)
bs_lp = BlackScholes(spot, long_put, 0.05, vol, days/365)
bs_sc = BlackScholes(spot, short_call, 0.05, vol, days/365)
bs_lc = BlackScholes(spot, long_call, 0.05, vol, days/365)

# Net credit
put_spread_credit = bs_sp.put_price() - bs_lp.put_price()
call_spread_credit = bs_sc.call_price() - bs_lc.call_price()
total_credit = (put_spread_credit + call_spread_credit) * 100
max_loss = (5 * 100) - total_credit

print(f"IRON CONDOR ANALYSIS: {symbol}")
print("=" * 50)
print(f"Current Price: ${spot:.2f}")
print(f"\nStructure:")
print(f"  Buy  ${long_put} Put")
print(f"  Sell ${short_put} Put")
print(f"  Sell ${short_call} Call")
print(f"  Buy  ${long_call} Call")
print(f"\nP&L:")
print(f"  Total Credit:  ${total_credit:.0f}")
print(f"  Max Loss:      ${max_loss:.0f}")
print(f"  Risk/Reward:   1:{total_credit/max_loss:.1f}")
print(f"  Break-even:    ${short_put - put_spread_credit:.0f} - ${short_call + call_spread_credit:.0f}")
print(f"  Profit Range:  ${short_put:.0f} - ${short_call:.0f} ({(short_call-short_put)/spot*100:.1f}% wide)")

---
## Summary

### Monthly Action Items

1. **Week 1**: Dollar-cost average into core ETFs (VTI/QQQ)
2. **Week 2-3**: Execute options strategies based on vol regime
3. **Week 4**: Review positions, roll expiring options

### Key Metrics to Track

- IV Percentile across watchlist
- P&L by strategy
- Win rate on options trades
- Portfolio theta (daily decay income)

### Next Steps

1. Set up Alpaca paper account for live testing
2. Implement automated screening alerts
3. Track results in portfolio tracker

In [ ]:
# Save this analysis
print("Analysis complete!")
print(f"\nTimestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Monthly investable: ${profile.monthly_investable:,.0f}")
print(f"Options budget: ${allocation.options_allocation:,.0f}")